In [1]:
import mysql.connector

connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="debi3"
)

cursor = connection.cursor()


In [2]:
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()
for table in tables:
    print(table)

('employee',)


In [5]:
# Insert multiple rows
employees = [
    (1, 'John Doe', 50000),
    (2, 'Jane Smith', 60000),
    (3, 'Mike Johnson', 70000)
]
cursor.executemany("INSERT INTO employee (id, name, salary) VALUES (%s, %s, %s)", employees)

In [6]:
# Select all records from a table
cursor.execute("SELECT * FROM employee")
records = cursor.fetchall()
for record in records:
    print(record)

(1, 'John Doe', None, Decimal('50000.00'))
(2, 'Jane Smith', None, Decimal('60000.00'))
(3, 'Mike Johnson', None, Decimal('70000.00'))


In [8]:
cursor.execute("SELECT * FROM employee WHERE salary >= 60000")
records = cursor.fetchall()
print(records)

[(2, 'Jane Smith', None, Decimal('60000.00')), (3, 'Mike Johnson', None, Decimal('70000.00'))]


# Comparison Operators
- = , <>

In [9]:
cursor.execute("SELECT * FROM employee WHERE salary <> 60000")
records = cursor.fetchall()
print(records)

[(1, 'John Doe', None, Decimal('50000.00')), (3, 'Mike Johnson', None, Decimal('70000.00'))]


In [10]:
cursor.execute("SELECT * FROM employee WHERE salary BETWEEN 50000 AND 70000")
records = cursor.fetchall()
print(records)

[(1, 'John Doe', None, Decimal('50000.00')), (2, 'Jane Smith', None, Decimal('60000.00')), (3, 'Mike Johnson', None, Decimal('70000.00'))]


In [11]:
cursor.execute("SELECT * FROM employee WHERE salary IN (50000, 60000)")
records = cursor.fetchall()
print(records)

[(1, 'John Doe', None, Decimal('50000.00')), (2, 'Jane Smith', None, Decimal('60000.00'))]


In [15]:
# select using ALL
cursor.execute("SELECT salary FROM employee WHERE salary = ALL (SELECT salary FROM employee WHERE name = 'Jane Smith')")
records = cursor.fetchall()
print(records)

[(Decimal('60000.00'),)]


In [16]:
# using LIKE
cursor.execute("SELECT * FROM employee WHERE name LIKE 'J%'")
records = cursor.fetchall()
print(records)

[(1, 'John Doe', None, Decimal('50000.00')), (2, 'Jane Smith', None, Decimal('60000.00'))]


In [17]:
# using LIKE
cursor.execute("SELECT * FROM employee WHERE name LIKE '_i%'")
records = cursor.fetchall()
print(records)

[(3, 'Mike Johnson', None, Decimal('70000.00'))]


Logical operators
- AND OR , NOT

In [18]:
cursor.execute("SELECT * FROM employee WHERE salary >= 60000 AND name LIKE 'J%'")
records = cursor.fetchall()
print(records)

[(2, 'Jane Smith', None, Decimal('60000.00'))]


Arithmetic Oprerations
- + , - , * , /

In [21]:
cursor.reset()

In [22]:
cursor.execute("SELECT * FROM employee WHERE salary >= 60000+1000")
records = cursor.fetchall()
print(records)

[(3, 'Mike Johnson', None, Decimal('70000.00'))]


# ORDER BY
- ASC
- DESC

In [23]:
cursor.execute("SELECT * FROM employee ORDER BY salary DESC")
records = cursor.fetchall()
print(records)

[(3, 'Mike Johnson', None, Decimal('70000.00')), (2, 'Jane Smith', None, Decimal('60000.00')), (1, 'John Doe', None, Decimal('50000.00'))]


In [24]:
cursor.execute("SELECT * FROM employee ORDER BY name ASC")
records = cursor.fetchall()
print(records)

[(2, 'Jane Smith', None, Decimal('60000.00')), (1, 'John Doe', None, Decimal('50000.00')), (3, 'Mike Johnson', None, Decimal('70000.00'))]


# Join

In [25]:
# Create tables for JOIN demonstration

# Create departments table
cursor.execute("""
CREATE TABLE IF NOT EXISTS departments (
    dept_id INT PRIMARY KEY,
    dept_name VARCHAR(50),
    location VARCHAR(50)
)
""")

# Create employees table with department reference
cursor.execute("""
CREATE TABLE IF NOT EXISTS employees (
    emp_id INT PRIMARY KEY,
    emp_name VARCHAR(50),
    salary DECIMAL(10,2),
    dept_id INT,
    hire_date DATE
)
""")

# Create projects table
cursor.execute("""
CREATE TABLE IF NOT EXISTS projects (
    project_id INT PRIMARY KEY,
    project_name VARCHAR(100),
    budget DECIMAL(12,2),
    dept_id INT
)
""")

# Create employee_projects table (many-to-many relationship)
cursor.execute("""
CREATE TABLE IF NOT EXISTS employee_projects (
    emp_id INT,
    project_id INT,
    role VARCHAR(50),
    PRIMARY KEY (emp_id, project_id)
)
""")

connection.commit()
print("Tables created successfully!")

Tables created successfully!


In [26]:
# Insert sample data into departments table
departments_data = [
    (1, 'Human Resources', 'New York'),
    (2, 'Engineering', 'San Francisco'),
    (3, 'Marketing', 'Chicago'),
    (4, 'Finance', 'New York'),
    (5, 'Research', 'Boston')
]

cursor.executemany("INSERT IGNORE INTO departments (dept_id, dept_name, location) VALUES (%s, %s, %s)", departments_data)
connection.commit()
print("Departments data inserted!")

Departments data inserted!


In [27]:
# Insert sample data into employees table
employees_data = [
    (101, 'Alice Johnson', 75000.00, 1, '2022-01-15'),
    (102, 'Bob Smith', 85000.00, 2, '2021-06-10'),
    (103, 'Charlie Brown', 65000.00, 3, '2023-03-20'),
    (104, 'Diana Prince', 90000.00, 2, '2020-09-05'),
    (105, 'Eve Adams', 70000.00, 4, '2022-11-12'),
    (106, 'Frank Miller', 95000.00, 2, '2019-04-18'),
    (107, 'Grace Lee', 60000.00, 3, '2023-07-01'),
    (108, 'Henry Wilson', 80000.00, 1, '2021-12-03'),
    (109, 'Ivy Chen', 120000.00, None, '2023-01-10')  # Employee without department
]

cursor.executemany("INSERT IGNORE INTO employees (emp_id, emp_name, salary, dept_id, hire_date) VALUES (%s, %s, %s, %s, %s)", employees_data)
connection.commit()
print("Employees data inserted!")

Employees data inserted!


In [28]:
# Insert sample data into projects table
projects_data = [
    (201, 'Website Redesign', 150000.00, 2),
    (202, 'HR System Upgrade', 80000.00, 1),
    (203, 'Marketing Campaign', 120000.00, 3),
    (204, 'Budget Analysis Tool', 95000.00, 4),
    (205, 'Mobile App Development', 200000.00, 2),
    (206, 'Data Analytics Platform', 250000.00, 5),
    (207, 'Social Media Strategy', 75000.00, 3),
    (208, 'Legacy System Migration', 300000.00, None)  # Project without assigned department
]

cursor.executemany("INSERT IGNORE INTO projects (project_id, project_name, budget, dept_id) VALUES (%s, %s, %s, %s)", projects_data)
connection.commit()
print("Projects data inserted!")

Projects data inserted!


In [29]:
# Insert sample data into employee_projects table
employee_projects_data = [
    (102, 201, 'Lead Developer'),
    (104, 201, 'Senior Developer'),
    (106, 201, 'Technical Lead'),
    (101, 202, 'Project Manager'),
    (108, 202, 'Business Analyst'),
    (103, 203, 'Marketing Specialist'),
    (107, 203, 'Content Creator'),
    (105, 204, 'Financial Analyst'),
    (102, 205, 'Mobile Developer'),
    (104, 205, 'Backend Developer'),
    (109, 206, 'Data Scientist'),
    (103, 207, 'Social Media Manager')
]

cursor.executemany("INSERT IGNORE INTO employee_projects (emp_id, project_id, role) VALUES (%s, %s, %s)", employee_projects_data)
connection.commit()
print("Employee-Projects relationships inserted!")

Employee-Projects relationships inserted!


In [30]:
# Verify the created tables and data
print("=== DEPARTMENTS ===")
cursor.execute("SELECT * FROM departments")
departments = cursor.fetchall()
for dept in departments:
    print(dept)

print("\n=== EMPLOYEES ===")
cursor.execute("SELECT * FROM employees")
employees = cursor.fetchall()
for emp in employees:
    print(emp)

print("\n=== PROJECTS ===")
cursor.execute("SELECT * FROM projects")
projects = cursor.fetchall()
for project in projects:
    print(project)

print("\n=== EMPLOYEE_PROJECTS ===")
cursor.execute("SELECT * FROM employee_projects")
emp_projects = cursor.fetchall()
for ep in emp_projects:
    print(ep)

=== DEPARTMENTS ===
(1, 'Human Resources', 'New York')
(2, 'Engineering', 'San Francisco')
(3, 'Marketing', 'Chicago')
(4, 'Finance', 'New York')
(5, 'Research', 'Boston')

=== EMPLOYEES ===
(101, 'Alice Johnson', Decimal('75000.00'), 1, datetime.date(2022, 1, 15))
(102, 'Bob Smith', Decimal('85000.00'), 2, datetime.date(2021, 6, 10))
(103, 'Charlie Brown', Decimal('65000.00'), 3, datetime.date(2023, 3, 20))
(104, 'Diana Prince', Decimal('90000.00'), 2, datetime.date(2020, 9, 5))
(105, 'Eve Adams', Decimal('70000.00'), 4, datetime.date(2022, 11, 12))
(106, 'Frank Miller', Decimal('95000.00'), 2, datetime.date(2019, 4, 18))
(107, 'Grace Lee', Decimal('60000.00'), 3, datetime.date(2023, 7, 1))
(108, 'Henry Wilson', Decimal('80000.00'), 1, datetime.date(2021, 12, 3))
(109, 'Ivy Chen', Decimal('120000.00'), None, datetime.date(2023, 1, 10))

=== PROJECTS ===
(201, 'Website Redesign', Decimal('150000.00'), 2)
(202, 'HR System Upgrade', Decimal('80000.00'), 1)
(203, 'Marketing Campaign', Dec